# Test compatibility gurobi
The idea is test if the model development is compatible with gurobi constraints. Using the packaege gurobi machine learning.

This code is hardcoded and connected with the model process b in this example

## Root folder and read env variables

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('\\')[:-2]
root_path = '\\'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  D:\github-mi-repo\Gurobi-ML-tips-modeling


In [2]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_GCP = os.environ.get("PROJECT_GCP", "")

## LOAD LICENCE GUROBI

In [3]:
##########  LOAD LICENCE GUROBI ##########
import gurobipy as gp

# set env variable with the path of the licence
name_file_licence_gurobi = "gurobi.lic"
path_licence_gurobi = root_path + '\\' + name_file_licence_gurobi
os.environ ["GRB_LICENSE_FILE"] = path_licence_gurobi
print(os.environ["GRB_LICENSE_FILE"])

D:\github-mi-repo\Gurobi-ML-tips-modeling\gurobi.lic


In [4]:
######### LAOD CONTENT LICENCE GUROBI #########
with open(path_licence_gurobi, 'r') as f:
    content_licence = f.read()
WLSACCESSID = content_licence.split('\n')[3].split('=')[1] # load WLSACCESSID (string)
WLSSECRET = content_licence.split('\n')[4].split('=')[1] # load WLSSECRET (string)
LICENSEID = int(content_licence.split('\n')[5].split("=")[1]) # load LICENSEID (integer)

params = {
"WLSACCESSID": WLSACCESSID,
"WLSSECRET": WLSSECRET,
"LICENSEID": LICENSEID
}

In [5]:
######### lOAD GUROBI MODEL #########
env = gp.Env(params=params)

#Create the model within the Gurobi environment
m_test_licence = gp.Model("Gurobi test model", env = env)
m_test_licence

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2441807
WLS license 2441807 - registered to CMPC Celulosa S.A


<gurobi.Model Continuous instance Gurobi test model: 0 constrs, 0 vars, Parameter changes: LicenseID=2441807>

## RUN

In [6]:
import pickle
import pandas as pd
import numpy as np

#gurobi
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr
import gurobipy as gp

### 0. Load data
This data will be use to get values to generate a instance of the ml model

In [7]:
name_process = 'process_b'
# load X_test
path_X_test = f'artifacts/data_training/{name_process}/X_test.pkl'
X_test = pd.read_pickle(path_X_test)

# load y_test
path_y_test = f'artifacts/data_training/{name_process}/y_test.pkl'
y_test = pd.read_pickle(path_y_test)

### 1. Load Artifacts to connect ML to gurobi

#### 1.1 pkl model

In [8]:
path_model_to_test = f'artifacts/models/{name_process}/lr.pkl'
model_ml_to_test = pd.read_pickle(path_model_to_test)
model_ml_to_test

Pipeline(steps=[('scaler', StandardScaler()),
                ('poly_feature_2', PolynomialFeatures()),
                ('lr', LinearRegression())])

### 1.2 Define list of features and target for each model

In [9]:
X_test

,Z1,X2,O4,O5
521,125.827767,1.600197,4.554699,0.088144
737,86.789512,0.578117,7.962496,3.903218
740,91.836144,6.023679,0.332634,2.712363
660,85.987786,3.098621,3.222811,0.905589
411,85.749421,5.509697,3.307710,4.956510
...,...,...,...,...
408,110.079863,6.547218,9.923618,8.871421
332,110.343326,3.887887,5.537505,4.707576
208,94.335957,7.043643,3.328544,0.851833
613,118.819866,2.913836,4.735953,0.224784


In [10]:
######################## model  ########################

list_features = ['Z1', 'X2', 'O4', 'O5']

list_features_controlables = ['Z1', 'X2']

list_target = ['Y2']

### 1.3 Read master tag and sort features according its order

In [11]:
# read table master tag
path_list_features_target_to_optimization = f'config/config_ml_models_development/MasterTable_{name_process}.xlsx'
maestro_tags = pd.read_excel(path_list_features_target_to_optimization)

### sort list of features according the order in master table
list_features = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features]
list_features_controlables = [tag for tag in maestro_tags['TAG'].tolist() if tag in list_features_controlables]

## 2. Create gurobi model

In [12]:
# create model
m = gp.Model('modelo',  env = env)

### 3. Create decision variables
- Decision variables that are features in ml models
- Decicion variable that is the output in ml models

In [13]:
# define set
list_set_elements_index = ['set_test']
index_set = pd.Index(list_set_elements_index)
index_set

Index(['set_test'], dtype='object')

In [14]:
list_target

['Y2']

In [15]:
# create decision variables - features ml model
var_Z1 = gppd.add_vars(m, index_set, name = "decision variable Z1"
                                     )

var_X2 = gppd.add_vars(m, index_set, name = "decision variable X2"
                                     )

In [16]:
# crete decision variables - output ml model
var_Y2 = gppd.add_vars(m, index_set, name = "decision variable Y2"
                                     )

In [17]:
# "compile"
m.update()

In [18]:
# see decision var created
var_Y2

set_test    <gurobi.Var decision variable Y2[set_test]>
Name: decision variable Y2, dtype: object

### 4. Create instance of Machine learning model using decision var of gurobi (decision var in optimization)

In [19]:
######################## generate instance NO controlables features for model ########################

# list feature NC
list_features_no_vc = list(set(list_features) - set(list_features_controlables))

# generate dataframe with the mean
instance_no_controlables = X_test[list_features_no_vc].mean().to_frame().T
instance_no_controlables

,O4,O5
0,5.216518,4.53373


In [20]:
######################## genrate instance - features no controlables + decision vars ########################

# create instance with controlables variables. sorted according the list of features. ES MUY IMPORTANTE QUE ESTÉ ORDENADO LAS VARIABLES DE DECUISIÓN DE ACUERDO A LA LISTA DE FEATURES
instance_controlables = pd.DataFrame([var_Z1, var_X2]).T # ADD DECISION VARIABLES
instance_controlables.columns = list_features_controlables # rename columns
instance_controlables.reset_index(inplace = True)
instance_controlables.drop(columns = 'index', inplace = True)

# append features controlables with no controlables
instance = pd.concat([instance_no_controlables, instance_controlables], axis = 1)
instance = instance[list_features] # sort features

# set index - optimization set
instance.index = index_set

In [21]:
instance

,Z1,X2,O4,O5
set_test,<gurobi.Var decision variable Z1[set_test]>,<gurobi.Var decision variable X2[set_test]>,5.216518,4.53373


In [22]:
###### load ml constraint ######
pred_constr = add_predictor_constr(gp_model = m, 
                                                    predictor = model_ml_to_test, 
                                                    input_vars = instance, # instance pandas gurobi
                                                    output_vars = var_Y2, # target
                                                    name = f'model_predict'
                                                   )
pred_constr.print_stats()

Model for model_predict:
21 variables
5 constraints
15 quadratic constraints
Input has shape (1, 4)
Output has shape (1, 1)

Pipeline has 3 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
std_scaler            (1, 4)            6            4            0            0

poly_feat            (1, 15)           15            0           15            0

lin_reg               (1, 1)            0            1            0            0

--------------------------------------------------------------------------------


#### NOTE IN THIS PART YOU CAN SEE IF THE MODEL CAN CONNNECT TO GUROBI!

### 5. Define objective optimization
Objetive that no generate infeasibility

In [23]:
m.setObjective(var_Y2.sum(),
               gp.GRB.MINIMIZE)

#### 6. Optimize and get optimal values

In [24]:
# solve
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19043.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

WLS license 2441807 - registered to CMPC Celulosa S.A
Optimize a model with 5 rows, 24 columns and 23 nonzeros
Model fingerprint: 0xabb844a1
Model has 15 quadratic constraints
Coefficient statistics:
  Matrix range     [4e-03, 2e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 5e+00]
  RHS range        [5e+00, 1e+02]
  QRHS range       [1e+00, 1e+00]
Presolve removed 4 rows and 7 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 4 rows and 19 columns
Presolve time: 0.00s
Presolved: 11 rows, 6 columns, 26 nonzeros
Presolved model has 3 bilinear constraint(s)
         in product terms.
         Presolve was not able to compute smalle

In [25]:
#### know the status of the model - 2 a optimal solution was founded
# docu: https://www.gurobi.com/documentation/current/refman/optimization_status_codes.html#sec:StatusCodes
m.Status

2

In [26]:
# get optimal values and save in a dataframe
######## create a dataframe with set as index
solution = pd.DataFrame(index = index_set)

######################## save optimal values - features of models (only the features) ########################

# model
solution["var_Z1"] = var_Z1.gppd.X
solution["var_X2"] = var_X2.gppd.X


######################## save optimal values - targets of models (some targets are features of the model of the next step) ########################
solution["var_Y2"] = var_Y2.gppd.X  # model


######################## # get value objetive function ########################
opt_objetive_function = m.ObjVal

In [27]:
# show value objetive function
opt_objetive_function

0.0

In [28]:
# show value decision variables
solution

,var_Z1,var_X2,var_Y2
set_test,4.156942,20.466896,0.0
